In [ ]:
#!pip install df2gspread

# Start Here
**Generate dataframe of normalized values**

Instructions: The sections from `Secondary Indicators` to `OSUS` have dropdown arrows you can use to hide the cells. These cells don't need to be run because they were pasted into `All in One`

The purpose of these sections are for debugging and visualization

In [ ]:
# Importing Libraries
import itertools
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gspread
import seaborn as sns
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
from scipy import integrate
from scipy import interpolate

# Authenticating your google account: use the same account that has the drive shared
auth.authenticate_user()
from google.colab import auth
auth.authenticate_user()


from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Pulling the data from the spreadsheet and moving it into a dataframe
wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/169n2zgG-2fvAhgrbeLBlaQ8vZXMPlQAik5P7yxjxvGA/edit#gid=830131643")

In [ ]:
sheet1 = wb.worksheet("Normalization")
data = sheet1.get_all_values()
df = pd.DataFrame(data)
#print(df)

In [ ]:
# without the column name
df = df.iloc[1:]
#print(df)

In [ ]:
# only take the xc column of the normalization table
df_xc = df.iloc[:, -4]
df_xc.replace('', np.nan, inplace=True)
df_xc = df_xc.dropna().reset_index(drop=True)
df_xc = df_xc.astype(float)
df_xc

0     0.945843
1     0.963758
2     0.977564
3     0.968603
4     0.635294
5     0.374674
6     0.798662
7     0.014773
8     0.772414
9     0.319672
10    0.965000
11    0.586957
12    1.000000
13    0.251716
14    0.861314
15    0.862903
16    0.263158
17    0.115385
18    0.930233
19    0.711538
20    0.279793
21    0.863636
Name: 9, dtype: float64

In [ ]:
# Define the membership functions
def membership_W(x):
    if x <= 0:
        return 1
    elif 0 < x <= 0.7:
        return 1 - x / 0.7
    else:
        return 0

def membership_M(x):
    if x < 0:
        return 0
    elif 0 <= x <= 0.7:
        return x / 0.7
    elif 0.7 < x <= 1:
        return (1 - x) / 0.3
    else:
        return 0

def membership_S(x):
    if x <= 0.7:
        return 0
    elif 0.7 < x <= 1:
        return (x - 0.7) / 0.3
    else:
        return 1

# Secondary Indicators

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
df_WMS = pd.DataFrame(columns=['W', 'M', 'S'])

for x in list(df_xc.values.astype(float)):
    W = membership_W(x)
    M = membership_M(x)
    S = membership_S(x)

    # Append a new row to df_WMS
    df_WMS = df_WMS.append({'W': W, 'M': M, 'S': S}, ignore_index=True)

print(df_WMS)


           W         M         S
0   0.000000  0.180524  0.819476
1   0.000000  0.120807  0.879193
2   0.000000  0.074786  0.925214
3   0.000000  0.104657  0.895343
4   0.092437  0.907563  0.000000
5   0.464752  0.535248  0.000000
6   0.000000  0.671126  0.328874
7   0.978896  0.021104  0.000000
8   0.000000  0.758621  0.241379
9   0.543326  0.456674  0.000000
10  0.000000  0.116667  0.883333
11  0.161491  0.838509  0.000000
12  0.000000  0.000000  1.000000
13  0.640405  0.359595  0.000000
14  0.000000  0.462287  0.537713
15  0.000000  0.456989  0.543011
16  0.624060  0.375940  0.000000
17  0.835165  0.164835  0.000000
18  0.000000  0.232558  0.767442
19  0.000000  0.961538  0.038462
20  0.600296  0.399704  0.000000
21  0.000000  0.454545  0.545455


In [ ]:
# Define the values for each column
WMS = ['W', 'M', 'S']

def generate_combinations(values, repeat):
    # Generate all combinations of values
    combinations = list(itertools.product(values, repeat=repeat))

    # Create a DataFrame
    df = pd.DataFrame(combinations)

    return df

In [ ]:
# Function to calculate SUM
def calculate_WMS_sum(row):
    return sum(row.map({'W': 0, 'M': 1, 'S': 2}))

# Create dataframes to store results
#ecos
df_air = generate_combinations(WMS, 3)
df_land = generate_combinations(WMS, 3)
df_water = generate_combinations(WMS, 3)
#hums
df_health = generate_combinations(WMS, 3)
df_polic = generate_combinations(WMS, 2)
df_know = generate_combinations(WMS, 2)
#wealth
df_wealth_pr = generate_combinations(WMS, 2)
df_wealth_st = generate_combinations(WMS, 2)
df_wealth_re = generate_combinations(WMS, 2)

dfs = [df_air, df_land, df_water, df_health, df_polic, df_know, df_wealth_pr, df_wealth_st, df_wealth_re]
df_names = ['df_air', 'df_land', 'df_water', 'df_health', 'df_polic', 'df_know', 'df_wealth_pr', 'df_wealth_st', 'df_wealth_re']

# Functions to determine Output based on SUM
def air_rules(row):
    if 0 <= row <= 1:
        return 'VB'
    elif 2 <= row <= 3:
        return 'B'
    elif row == 4:
        return 'A'
    elif row == 5:
        return 'G'
    elif row == 6:
        return 'VG'

def land_rules(row):
    if 0 <= row <= 1:
        return 'VB'
    elif 2 <= row <= 3:
        return 'B'
    elif row == 4:
        return 'A'
    elif row == 5:
        return 'G'
    elif row == 6:
        return 'VG'

def water_rules(row):
    if 0 <= row <= 1:
        return 'VB'
    elif row == 2:
        return 'B'
    elif row == 3:
        return 'A'
    elif row == 4:
        return 'G'
    elif 5 <= row <= 6:
        return 'VG'

def health_rules(row):
    if 0 <= row <= 1:
        return 'VB'
    elif 2 <= row <= 3:
        return 'B'
    elif row == 4:
        return 'A'
    elif row == 5:
        return 'G'
    elif row == 6:
        return 'VG'

def polic_rules(row):
    if row == 0:
        return 'VB'
    elif row == 1:
        return 'B'
    elif row == 2:
        return 'A'
    elif row == 3:
        return 'G'
    elif row == 4:
        return 'VG'

def know_rules(row):
    if row == 0:
        return 'VB'
    elif row == 1:
        return 'B'
    elif row == 2:
        return 'A'
    elif row == 3:
        return 'G'
    elif row == 4:
        return 'VG'

def wealth_pr_rules(row):
        if row == 0:
            return 'VB'
        elif row == 1:
            return 'B'
        elif row == 2:
            return 'A'
        elif row == 3:
            return 'G'
        elif row == 4:
            return 'VG'

def wealth_st_rules(row):
    if row == 0:
        return 'VB'
    elif row == 1:
        return 'B'
    elif row == 2:
        return 'A'
    elif row == 3:
        return 'G'
    elif row == 4:
        return 'VG'

def wealth_re_rules(row):
    if row == 0:
        return 'VB'
    elif row == 1:
        return 'B'
    elif row == 2:
        return 'A'
    elif row == 3:
        return 'G'
    elif row == 4:
        return 'VG'

for df in dfs:
    # Apply the function to create the 'SUM' column
    df['SUM'] = df.apply(calculate_WMS_sum, axis=1)

# Apply the function to create the 'Output' column
df_air['Output'] = df_air['SUM'].apply(air_rules)
df_land['Output'] = df_land['SUM'].apply(land_rules)
df_water['Output'] = df_water['SUM'].apply(water_rules)
df_health['Output'] = df_health['SUM'].apply(health_rules)
df_polic['Output'] = df_polic['SUM'].apply(polic_rules)
df_know['Output'] = df_know['SUM'].apply(know_rules)
df_wealth_pr['Output'] = df_wealth_pr['SUM'].apply(wealth_pr_rules)
df_wealth_st['Output'] = df_wealth_st['SUM'].apply(wealth_st_rules)
df_wealth_re['Output'] = df_wealth_re['SUM'].apply(wealth_re_rules)


# Display the resulting DataFrame
# print(df_wealth_pr)
# print(df_wealth_st)
# print(df_wealth_re)

In [ ]:
def calculate_secondary(fuzzy_input, rule_base):
  # Iterate through each rule in the rule base
  terms = max(len(value) for value in fuzzy_input.values())
  for i in range(terms):
    # Extract the values for each linguistic variable from the rule
    w_value = fuzzy_input['W'][i]
    m_value = fuzzy_input['M'][i]
    s_value = fuzzy_input['S'][i]

    # Replace the linguistic values in the rule_base DataFrame
    rule_base.iloc[:,i] = rule_base.iloc[:,i].replace({'W': w_value, 'M': m_value, 'S': s_value})

  # Multiply the fuzzy input values across the row to get the rule result
  rule_base['Product'] = rule_base.iloc[:, :terms].apply(lambda row: row.product(), axis=1)
  # print(rule_base)

  nonzero_products = rule_base[rule_base['Product'] != 0]['Product']

  # Extract the values assigned to VB, B, A, VG, G
  VB = nonzero_products[rule_base['Output'] == 'VB'].sum()
  B = nonzero_products[rule_base['Output'] == 'B'].sum()
  A = nonzero_products[rule_base['Output'] == 'A'].sum()
  G = nonzero_products[rule_base['Output'] == 'G'].sum()
  VG = nonzero_products[rule_base['Output'] == 'VG'].sum()

  # Sum the values
  crisp_value = (0*VB + 0.25*B + 0.5*A + 0.75*G + 1*VG) / (VB + B + A + G + VG)
  return [crisp_value,[VB, B, A, G, VG]]

In [ ]:
# Extract Fuzzy Inputs from df_WMS
air_WMS = df_WMS[:3]
land_WMS = df_WMS[3:6]
land_WMS.reset_index(drop=True, inplace=True)
water_WMS = df_WMS[6:9]
water_WMS.reset_index(drop=True, inplace=True)
polic_WMS = df_WMS[9:11]
polic_WMS.reset_index(drop=True, inplace=True)
wealth_pr_WMS = df_WMS[11:13]
wealth_pr_WMS.reset_index(drop=True, inplace=True)
wealth_st_WMS = df_WMS[13:15]
wealth_st_WMS.reset_index(drop=True, inplace=True)
wealth_re_WMS = df_WMS[15:17]
wealth_re_WMS.reset_index(drop=True, inplace=True)
health_WMS = df_WMS[17:20]
health_WMS.reset_index(drop=True, inplace=True)
know_WMS = df_WMS[20:22]
know_WMS.reset_index(drop=True, inplace=True)

air_fuzzy_input = dict(air_WMS)
land_fuzzy_input = dict(land_WMS)
water_fuzzy_input = dict(water_WMS)
polic_fuzzy_input = dict(polic_WMS)
health_fuzzy_input = dict(health_WMS)
know_fuzzy_input = dict(know_WMS)
wealth_pr_fuzzy_input = dict(wealth_pr_WMS)
wealth_st_fuzzy_input = dict(wealth_st_WMS)
wealth_re_fuzzy_input = dict(wealth_re_WMS)

In [ ]:
air_crisp_value, air_fuzzies = calculate_secondary(air_fuzzy_input, df_air)
print(air_crisp_value)
print(air_fuzzies)

0.905970680268
[0.0, 0.0016309826595332616, 0.03945105475919975, 0.29232222143100106, 0.6665957411502664]


In [ ]:
land_crisp_value, land_fuzzies = calculate_secondary(land_fuzzy_input, df_land)
print(land_crisp_value)
print(land_fuzzies)

0.35760893460188226
[0.004496103092194456, 0.5605720554080822, 0.4349318414997235, 0.0, 0.0]


In [ ]:
water_crisp_value, water_fuzzies = calculate_secondary(water_fuzzy_input, df_water)
print(water_crisp_value)
print(water_fuzzies)

0.39783930774371423
[0.0, 0.4983854226733044, 0.41354722239281044, 0.08639205621960988, 0.001675298714275702]


In [ ]:
polic_crisp_value, polic_fuzzies = calculate_secondary(polic_fuzzy_input, df_polic)
print(polic_crisp_value)
print(polic_fuzzies)

0.5850019516002977
[0.0, 0.0633879781420834, 0.533216237314643, 0.40339578454327385, 0.0]


In [ ]:
health_crisp_value, health_fuzzies = calculate_secondary(health_fuzzy_input, df_health)
print(health_crisp_value)
print(health_fuzzies)

0.28937319389020444
[0.0, 0.8473726631150119, 0.1477618982091592, 0.004865438675829191, 0.0]


In [ ]:
know_crisp_value, know_fuzzies = calculate_secondary(know_fuzzy_input, df_know)
print(know_crisp_value)
print(know_fuzzies)

0.4862896171186667
[0.0, 0.2728618531726546, 0.5091178251800244, 0.21802032164732119, 0.0]


In [ ]:
wealth_pr_crisp_value, wealth_pr_fuzzies = calculate_secondary(wealth_pr_fuzzy_input, df_wealth_pr)
print(wealth_pr_crisp_value)
print(wealth_pr_fuzzies)

0.7096273291925358
[0.0, 0.0, 0.16149068322985718, 0.8385093167701431, 0.0]


In [ ]:
wealth_st_crisp_value, wealth_st_fuzzies = calculate_secondary(wealth_st_fuzzy_input, df_wealth_st)
print(wealth_st_crisp_value)
print(wealth_st_fuzzies)

0.4743268835369643
[0.0, 0.2960511402276669, 0.5105901853968092, 0.1933586743755241, 0.0]


In [ ]:
wealth_re_crisp_value, wealth_re_fuzzies = calculate_secondary(wealth_re_fuzzy_input, df_wealth_re)
print(wealth_re_crisp_value)
print(wealth_re_fuzzies)

0.4797376505780834
[0.0, 0.28518877839753365, 0.5106718408925993, 0.20413938070986723, 0.0]


In [ ]:
for i, df in enumerate(dfs):
    # Drop rows that do not fire
    dfs[i] = df[df['Product'] != 0]
    print(f"{df_names[i]}:\n {dfs[i]}\n")

df_air:
            0         1         2  SUM Output   Product
13  0.180524  0.120807  0.074786    3      B  0.001631
14  0.180524  0.120807  0.925214    4      A  0.020178
16  0.180524  0.879193  0.074786    4      A  0.011870
17  0.180524  0.879193  0.925214    5      G  0.146845
22  0.819476  0.120807  0.074786    4      A  0.007404
23  0.819476  0.120807  0.925214    5      G  0.091595
25  0.819476  0.879193  0.074786    5      G  0.053882
26  0.819476  0.879193  0.925214    6     VG  0.666596

df_land:
            0         1         2  SUM Output   Product
9   0.104657  0.092437  0.464752    1     VB  0.004496
10  0.104657  0.092437  0.535248    2      B  0.005178
12  0.104657  0.907563  0.464752    2      B  0.044144
13  0.104657  0.907563  0.535248    3      B  0.050839
18  0.895343  0.092437  0.464752    2      B  0.038464
19  0.895343  0.092437  0.535248    3      B  0.044299
21  0.895343  0.907563  0.464752    3      B  0.377648
22  0.895343  0.907563  0.535248    4      A 

# ECOS and HUMS

In [ ]:
# Create dataframes to store results
VBBAGVG = ['VB', 'B', 'A', 'G', 'VG']
df_ecos = generate_combinations(VBBAGVG, 3)
df_hums = generate_combinations(VBBAGVG, 3)
df_wealth = generate_combinations(VBBAGVG, 3)

dfs = [df_ecos, df_hums, df_wealth]

# we want to weigh everything the same in wealth tho
def calculate_sum(row):
    return sum(row.map({'VB': 0, 'B': 1, 'A': 2, 'G': 3, 'VG': 4}))

def calculate_custom_sum(row):
    col1_sum = {'VB': 0, 'B': 2, 'A': 4, 'G': 6, 'VG': 8}
    col2_to_n_sum = {'VB': 0, 'B': 1, 'A': 2, 'G': 3, 'VG': 4}
    return col1_sum[row[0]] + sum(col2_to_n_sum[val] for val in row[1:])


def ecos_rules(row):
    if 0 <= row <= 3:
        return 'VB'
    elif 4 <= row <= 7:
        return 'B'
    elif 8 <= row <= 11:
        return 'A'
    elif 12 <= row <= 14:
        return 'G'
    elif 15 <= row <= 16:
        return 'VG'

def hums_rules(row):
    if 0 <= row <= 3:
        return 'VB'
    elif 4 <= row <= 7:
        return 'B'
    elif 8 <= row <= 11:
        return 'A'
    elif 12 <= row <= 14:
        return 'G'
    elif 15 <= row <= 16:
        return 'VG'

def wealth_rules(row):
    if 0 <= row <= 2:
        return 'VB'
    elif 3 <= row <= 5:
        return 'B'
    elif 6 <= row <= 8:
        return 'A'
    elif 9 <= row <= 10:
        return 'G'
    elif 11 <= row <= 12:
        return 'VG'

for df in dfs:
  if df is df_wealth:
      df['SUM'] = df.apply(calculate_sum, axis=1)
  else:
      df['SUM'] = df.apply(calculate_custom_sum, axis=1)

df_ecos['Output'] = df_ecos['SUM'].apply(ecos_rules)
df_hums['Output'] = df_hums['SUM'].apply(hums_rules)
df_wealth['Output'] = df_wealth['SUM'].apply(wealth_rules)

In [ ]:
df_ecos

,0,1,2,SUM,Output
0,VB,VB,VB,0,VB
1,VB,VB,B,1,VB
2,VB,VB,A,2,VB
3,VB,VB,G,3,VB
4,VB,VB,VG,4,B
...,...,...,...,...,...
120,VG,VG,VB,12,G
121,VG,VG,B,13,G
122,VG,VG,A,14,G
123,VG,VG,G,15,VG


In [ ]:
df_hums

,0,1,2,SUM,Output
0,VB,VB,VB,0,VB
1,VB,VB,B,1,VB
2,VB,VB,A,2,VB
3,VB,VB,G,3,VB
4,VB,VB,VG,4,B
...,...,...,...,...,...
120,VG,VG,VB,12,G
121,VG,VG,B,13,G
122,VG,VG,A,14,G
123,VG,VG,G,15,VG


In [ ]:
df_wealth

,0,1,2,SUM,Output
0,VB,VB,VB,0,VB
1,VB,VB,B,1,VB
2,VB,VB,A,2,VB
3,VB,VB,G,3,B
4,VB,VB,VG,4,B
...,...,...,...,...,...
120,VG,VG,VB,8,A
121,VG,VG,B,9,G
122,VG,VG,A,10,G
123,VG,VG,G,11,VG


In [ ]:
ecos_data = [air_fuzzies, land_fuzzies, water_fuzzies]
hums_data = [polic_fuzzies, health_fuzzies, know_fuzzies]
wealth_data = [wealth_pr_fuzzies, wealth_st_fuzzies, wealth_re_fuzzies]

ecos_ling = pd.DataFrame(ecos_data, columns=VBBAGVG)
hums_ling = pd.DataFrame(hums_data, columns=VBBAGVG)
wealth_ling = pd.DataFrame(wealth_data, columns=VBBAGVG)

ecos_fuzzy_input = dict(ecos_ling)
hums_fuzzy_input = dict(hums_ling)
wealth_fuzzy_input = dict(wealth_ling)

In [ ]:
ecos_ling

,VB,B,A,G,VG
0,0.000000,0.001631,0.039451,0.292322,0.666596
1,0.004496,0.560572,0.434932,0.000000,0.000000
2,0.000000,0.498385,0.413547,0.086392,0.001675


In [ ]:
def calculate_primary(fuzzy_input, rule_base):
  # Iterate through each rule in the rule base
  terms = max(len(value) for value in fuzzy_input.values())
  for i in range(terms):
    # Extract the values for each linguistic variable from the rule
    VB_value = fuzzy_input['VB'][i]
    B_value = fuzzy_input['B'][i]
    A_value = fuzzy_input['A'][i]
    G_value = fuzzy_input['G'][i]
    VG_value = fuzzy_input['VG'][i]

    # Replace the linguistic values in the rule_base DataFrame
    rule_base.iloc[:,i] = rule_base.iloc[:,i].replace({'VB': VB_value, 'B': B_value, 'A': A_value, 'G': G_value, 'VG': VG_value})

  # Multiply the fuzzy input values across the row to get the rule result
  rule_base['Product'] = rule_base.iloc[:, :terms].apply(lambda row: row.product(), axis=1)
  #print(rule_base)

  nonzero_products = rule_base[rule_base['Product'] != 0]['Product']

  # Extract the values assigned to VB, B, A, VG, G
  VB = nonzero_products[rule_base['Output'] == 'VB'].sum()
  B = nonzero_products[rule_base['Output'] == 'B'].sum()
  A = nonzero_products[rule_base['Output'] == 'A'].sum()
  G = nonzero_products[rule_base['Output'] == 'G'].sum()
  VG = nonzero_products[rule_base['Output'] == 'VG'].sum()

  # Sum the values
  crisp_value = (0*VB + 0.25*B + 0.5*A + 0.75*G + 1*VG) / (VB + B + A + G + VG)
  return [crisp_value,[VB, B, A, G, VG]]

In [ ]:
ecos_crisp_value, ecos_fuzzies = calculate_primary(ecos_fuzzy_input, df_ecos)
print(ecos_fuzzies)
print(ecos_crisp_value)

[3.6546932870339985e-06, 0.031177343283373925, 0.7902624582700573, 0.1785565437532829, 0.0]
0.5368429727708337


In [ ]:
hums_crisp_value, hums_fuzzies = calculate_primary(hums_fuzzy_input, df_hums)
print(hums_fuzzies)
print(hums_crisp_value)

[0.0, 0.4381433797125742, 0.5614287123576922, 0.00042790792973417365, 0.0]
0.39057113205429


In [ ]:
wealth_crisp_value, wealth_fuzzies = calculate_primary(wealth_fuzzy_input, df_wealth)
print(wealth_fuzzies)
print(wealth_crisp_value)

[0.0, 0.1323608286915149, 0.8345414309006828, 0.03309774040780294, 0.0]
0.475184227929072


In [ ]:
for i, df in enumerate(dfs):
    # Drop rows that do not fire
    dfs[i] = df[df['Product'] != 0]
    print(f"{dfs[i]}\n")

            0         1         2  SUM Output       Product
26   0.001631  0.004496  0.498385    3     VB  3.654693e-06
27   0.001631  0.004496  0.413547    4      B  3.032569e-06
28   0.001631  0.004496  0.086392    5      B  6.335187e-07
29   0.001631  0.004496  0.001675    6      B  1.228508e-08
31   0.001631  0.560572  0.498385    4      B  4.556655e-04
32   0.001631  0.560572  0.413547    5      B  3.780993e-04
33   0.001631  0.560572  0.086392    6      B  7.898681e-05
34   0.001631  0.560572  0.001675    7      B  1.531698e-06
36   0.001631  0.434932  0.498385    5      B  3.535378e-04
37   0.001631  0.434932  0.413547    6      B  2.933565e-04
38   0.001631  0.434932  0.086392    7      B  6.128361e-05
39   0.001631  0.434932  0.001675    8      A  1.188400e-06
51   0.039451  0.004496  0.498385    5      B  8.840162e-05
52   0.039451  0.004496  0.413547    6      B  7.335336e-05
53   0.039451  0.004496  0.086392    7      B  1.532388e-05
54   0.039451  0.004496  0.001675    8  

#OSUS

In [ ]:
df_osus = generate_combinations(VBBAGVG, 3) #
# rule base when wealth gets added
def osus_rules(row):
    if 0 <= row <= 1:
        return 'EL'
    elif 2 <= row <= 3:
        return 'VL'
    elif row == 4:
        return 'L'
    elif row == 5:
        return 'FL'
    elif row == 6:
        return 'I'
    elif row == 7:
        return 'FH'
    elif row == 8:
        return 'H'
    elif 9 <= row <= 10:
        return 'VH'
    elif 11<= row <= 12:
        return 'EH'

def calculate_osus_sum(row):
    return sum(row.map({'VB': 0, 'B': 1, 'A': 2, 'G': 3, 'VG': 4}))

df_osus['SUM'] = df_osus.apply(calculate_osus_sum, axis=1)

# Apply the function to create the 'Output' column
df_osus['Output'] = df_osus['SUM'].apply(osus_rules)
print(df_osus)

      0   1   2  SUM Output
0    VB  VB  VB    0     EL
1    VB  VB   B    1     EL
2    VB  VB   A    2     VL
3    VB  VB   G    3     VL
4    VB  VB  VG    4      L
..   ..  ..  ..  ...    ...
120  VG  VG  VB    8      H
121  VG  VG   B    9     VH
122  VG  VG   A   10     VH
123  VG  VG   G   11     EH
124  VG  VG  VG   12     EH

[125 rows x 5 columns]


In [ ]:
def calculate_osus(fuzzy_input, rule_base):
  # Iterate through each rule in the rule base
  terms = max(len(value) for value in fuzzy_input.values())
  for i in range(terms):
    # Extract the values for each linguistic variable from the rule
    VB_value = fuzzy_input['VB'][i]
    B_value = fuzzy_input['B'][i]
    A_value = fuzzy_input['A'][i]
    G_value = fuzzy_input['G'][i]
    VG_value = fuzzy_input['VG'][i]

    # Replace the linguistic values in the rule_base DataFrame
    rule_base.iloc[:,i] = rule_base.iloc[:,i].replace({'VB': VB_value, 'B': B_value, 'A': A_value, 'G': G_value, 'VG': VG_value})

  # Multiply the fuzzy input values across the row to get the rule result
  rule_base['Product'] = rule_base.iloc[:, :terms].apply(lambda row: row.product(), axis=1)
  #print(rule_base)

  nonzero_products = rule_base[rule_base['Product'] != 0]['Product']

  # Extract the values assigned to VB, B, A, VG, G
  EL = nonzero_products[rule_base['Output'] == 'EL'].sum()
  VL = nonzero_products[rule_base['Output'] == 'VL'].sum()
  L = nonzero_products[rule_base['Output'] == 'L'].sum()
  FL = nonzero_products[rule_base['Output'] == 'FL'].sum()
  I = nonzero_products[rule_base['Output'] == 'I'].sum()
  FH = nonzero_products[rule_base['Output'] == 'FH'].sum()
  H = nonzero_products[rule_base['Output'] == 'H'].sum()
  VH = nonzero_products[rule_base['Output'] == 'VH'].sum()
  EH = nonzero_products[rule_base['Output'] == 'EH'].sum()

  # Sum the values
  crisp_value = (0*EL + 0.125*VL + 0.25*L + 0.375*FL + 0.5*I + 0.625*FH + 0.75*H + 0.875*VH + 1*EH) / (EL + VL + L + FL + I + FH + H + VH + EH)
  return [crisp_value,[EL, VL, L, FL, I, FH, H, VH, EH]]

In [ ]:
osus_data = [ecos_fuzzies, hums_fuzzies, wealth_fuzzies] # need to add wealth

osus_ling = pd.DataFrame(osus_data, columns=VBBAGVG)

osus_fuzzy_input = dict(osus_ling)

In [ ]:
osus_crisp_value, osus_fuzzies = calculate_osus(osus_fuzzy_input, df_osus)
print(osus_crisp_value)

0.45129919287043596


In [ ]:
df_osus = df_osus[df_osus['Product'] != 0]
df_osus

,0,1,2,SUM,Output,Product
6,0.000004,0.438143,0.132361,2,VL,2.119467e-07
7,0.000004,0.438143,0.834541,3,VL,1.336334e-06
8,0.000004,0.438143,0.033098,4,L,5.299874e-08
11,0.000004,0.561429,0.132361,3,VL,2.715845e-07
12,0.000004,0.561429,0.834541,4,L,1.712354e-06
13,0.000004,0.561429,0.033098,5,FL,6.791159e-08
16,0.000004,0.000428,0.132361,4,L,2.069954e-10
17,0.000004,0.000428,0.834541,5,FL,1.305116e-09
18,0.000004,0.000428,0.033098,6,I,5.176064e-11
31,0.031177,0.438143,0.132361,3,VL,1.808068e-03


# All in One

In [ ]:
def get_osus(df_xc):

    # supress the warning
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=DeprecationWarning)

    df_WMS = pd.DataFrame(columns=['W', 'M', 'S'])

    for x in list(df_xc.values.astype(float)):
        W = membership_W(x)
        M = membership_M(x)
        S = membership_S(x)
        df_WMS = df_WMS.append({'W': W, 'M': M, 'S': S}, ignore_index=True)

    WMS = ['W', 'M', 'S']

    def generate_combinations(values, repeat):
        combinations = list(itertools.product(values, repeat=repeat))
        df = pd.DataFrame(combinations)
        return df

    def calculate_WMS_sum(row):
        return sum(row.map({'W': 0, 'M': 1, 'S': 2}))

    # ecos
    df_air = generate_combinations(WMS, 3)
    df_land = generate_combinations(WMS, 3)
    df_water = generate_combinations(WMS, 3)
    # hum
    df_health = generate_combinations(WMS, 3)
    df_polic = generate_combinations(WMS, 2)
    df_know = generate_combinations(WMS, 2)
    #wealth
    df_wealth_pr = generate_combinations(WMS, 2)
    df_wealth_st = generate_combinations(WMS, 2)
    df_wealth_re = generate_combinations(WMS, 2)

    dfs = [df_air, df_land, df_water, df_health, df_polic, df_know, df_wealth_pr, df_wealth_st, df_wealth_re]

    def air_rules(row):
        if 0 <= row <= 1:
            return 'VB'
        elif 2 <= row <= 3:
            return 'B'
        elif row == 4:
            return 'A'
        elif row == 5:
            return 'G'
        elif row == 6:
            return 'VG'

    def land_rules(row):
        if 0 <= row <= 1:
            return 'VB'
        elif 2 <= row <= 3:
            return 'B'
        elif row == 4:
            return 'A'
        elif row == 5:
            return 'G'
        elif row == 6:
            return 'VG'

    def water_rules(row):
        if 0 <= row <= 1:
            return 'VB'
        elif row == 2:
            return 'B'
        elif row == 3:
            return 'A'
        elif row == 4:
            return 'G'
        elif 5 <= row <= 6:
            return 'VG'

    def health_rules(row):
        if 0 <= row <= 1:
            return 'VB'
        elif 2 <= row <= 3:
            return 'B'
        elif row == 4:
            return 'A'
        elif row == 5:
            return 'G'
        elif row == 6:
            return 'VG'

    def polic_rules(row):
        if row == 0:
            return 'VB'
        elif row == 1:
            return 'B'
        elif row == 2:
            return 'A'
        elif row == 3:
            return 'G'
        elif row == 4:
            return 'VG'

    def know_rules(row):
        if row == 0:
            return 'VB'
        elif row == 1:
            return 'B'
        elif row == 2:
            return 'A'
        elif row == 3:
            return 'G'
        elif row == 4:
            return 'VG'

    def wealth_pr_rules(row):
        if row == 0:
            return 'VB'
        elif row == 1:
            return 'B'
        elif row == 2:
            return 'A'
        elif row == 3:
            return 'G'
        elif row == 4:
            return 'VG'

    def wealth_st_rules(row):
        if row == 0:
            return 'VB'
        elif row == 1:
            return 'B'
        elif row == 2:
            return 'A'
        elif row == 3:
            return 'G'
        elif row == 4:
            return 'VG'

    def wealth_re_rules(row):
        if row == 0:
            return 'VB'
        elif row == 1:
            return 'B'
        elif row == 2:
            return 'A'
        elif row == 3:
            return 'G'
        elif row == 4:
            return 'VG'

    for df in dfs:
        df['SUM'] = df.apply(calculate_WMS_sum, axis=1)

    df_air['Output'] = df_air['SUM'].apply(air_rules)
    df_land['Output'] = df_land['SUM'].apply(land_rules)
    df_water['Output'] = df_water['SUM'].apply(water_rules)
    df_health['Output'] = df_health['SUM'].apply(health_rules)
    df_polic['Output'] = df_polic['SUM'].apply(polic_rules)
    df_know['Output'] = df_know['SUM'].apply(know_rules)
    df_wealth_pr['Output'] = df_wealth_pr['SUM'].apply(wealth_pr_rules)
    df_wealth_st['Output'] = df_wealth_st['SUM'].apply(wealth_st_rules)
    df_wealth_re['Output'] = df_wealth_re['SUM'].apply(wealth_re_rules)

    def calculate_secondary(fuzzy_input, rule_base):
      # Iterate through each rule in the rule base
      terms = max(len(value) for value in fuzzy_input.values())
      for i in range(terms):
        # Extract the values for each linguistic variable from the rule
        w_value = fuzzy_input['W'][i]
        m_value = fuzzy_input['M'][i]
        s_value = fuzzy_input['S'][i]

        # Replace the linguistic values in the rule_base DataFrame
        rule_base.iloc[:,i] = rule_base.iloc[:,i].replace({'W': w_value, 'M': m_value, 'S': s_value})

      # Multiply the fuzzy input values across the row to get the rule result
      rule_base['Product'] = rule_base.iloc[:, :terms].apply(lambda row: row.product(), axis=1)
      #print(rule_base)

      nonzero_products = rule_base[rule_base['Product'] != 0]['Product']

      # Extract the values assigned to VB, B, A, VG, G
      VB = nonzero_products[rule_base['Output'] == 'VB'].sum()
      B = nonzero_products[rule_base['Output'] == 'B'].sum()
      A = nonzero_products[rule_base['Output'] == 'A'].sum()
      G = nonzero_products[rule_base['Output'] == 'G'].sum()
      VG = nonzero_products[rule_base['Output'] == 'VG'].sum()

      # Sum the values
      crisp_value = (0*VB + 0.25*B + 0.5*A + 0.75*G + 1*VG) / (VB + B + A + G + VG)
      return [crisp_value,[VB, B, A, G, VG]]

    air_WMS = df_WMS[:3]
    land_WMS = df_WMS[3:6]
    land_WMS.reset_index(drop=True, inplace=True)
    water_WMS = df_WMS[6:9]
    water_WMS.reset_index(drop=True, inplace=True)
    polic_WMS = df_WMS[9:11]
    polic_WMS.reset_index(drop=True, inplace=True)
    wealth_pr_WMS = df_WMS[11:13]
    wealth_pr_WMS.reset_index(drop=True, inplace=True)
    wealth_st_WMS = df_WMS[13:15]
    wealth_st_WMS.reset_index(drop=True, inplace=True)
    wealth_re_WMS = df_WMS[15:17]
    wealth_re_WMS.reset_index(drop=True, inplace=True)
    health_WMS = df_WMS[17:20]
    health_WMS.reset_index(drop=True, inplace=True)
    know_WMS = df_WMS[20:22]
    know_WMS.reset_index(drop=True, inplace=True)

    air_fuzzy_input = dict(air_WMS)
    land_fuzzy_input = dict(land_WMS)
    water_fuzzy_input = dict(water_WMS)
    polic_fuzzy_input = dict(polic_WMS)
    health_fuzzy_input = dict(health_WMS)
    know_fuzzy_input = dict(know_WMS)
    wealth_pr_fuzzy_input = dict(wealth_pr_WMS)
    wealth_st_fuzzy_input = dict(wealth_st_WMS)
    wealth_re_fuzzy_input = dict(wealth_re_WMS)

    air_crisp_value, air_fuzzies = calculate_secondary(air_fuzzy_input, df_air)
    land_crisp_value, land_fuzzies = calculate_secondary(land_fuzzy_input, df_land)
    water_crisp_value, water_fuzzies = calculate_secondary(water_fuzzy_input, df_water)
    polic_crisp_value, polic_fuzzies = calculate_secondary(polic_fuzzy_input, df_polic)
    health_crisp_value, health_fuzzies = calculate_secondary(health_fuzzy_input, df_health)
    know_crisp_value, know_fuzzies = calculate_secondary(know_fuzzy_input, df_know)
    wealth_pr_crisp_value, wealth_pr_fuzzies = calculate_secondary(wealth_pr_fuzzy_input, df_wealth_pr)
    wealth_st_crisp_value, wealth_st_fuzzies = calculate_secondary(wealth_st_fuzzy_input, df_wealth_st)
    wealth_re_crisp_value, wealth_re_fuzzies = calculate_secondary(wealth_re_fuzzy_input, df_wealth_re)

    for i, df in enumerate(dfs):
        dfs[i] = df[df['Product'] != 0]

    VBBAGVG = ['VB', 'B', 'A', 'G', 'VG']
    df_ecos = generate_combinations(VBBAGVG, 3)
    df_hums = generate_combinations(VBBAGVG, 3)
    df_wealth = generate_combinations(VBBAGVG, 3)

    dfs = [df_ecos, df_hums, df_wealth]

    # we want to weigh everything the same in wealth tho
    def calculate_sum(row):
        return sum(row.map({'VB': 0, 'B': 1, 'A': 2, 'G': 3, 'VG': 4}))

    def calculate_custom_sum(row):
        col1_sum = {'VB': 0, 'B': 2, 'A': 4, 'G': 6, 'VG': 8}
        col2_to_n_sum = {'VB': 0, 'B': 1, 'A': 2, 'G': 3, 'VG': 4}
        return col1_sum[row[0]] + sum(col2_to_n_sum[val] for val in row[1:])


    def ecos_rules(row):
        if 0 <= row <= 3:
            return 'VB'
        elif 4 <= row <= 7:
            return 'B'
        elif 8 <= row <= 11:
            return 'A'
        elif 12 <= row <= 14:
            return 'G'
        elif 15 <= row <= 16:
            return 'VG'

    def hums_rules(row):
        if 0 <= row <= 3:
            return 'VB'
        elif 4 <= row <= 7:
            return 'B'
        elif 8 <= row <= 11:
            return 'A'
        elif 12 <= row <= 14:
            return 'G'
        elif 15 <= row <= 16:
            return 'VG'

    def wealth_rules(row):
        if 0 <= row <= 2:
            return 'VB'
        elif 3 <= row <= 5:
            return 'B'
        elif 6 <= row <= 8:
            return 'A'
        elif 9 <= row <= 10:
            return 'G'
        elif 11 <= row <= 12:
            return 'VG'

    for df in dfs:
      if df is df_wealth:
          df['SUM'] = df.apply(calculate_sum, axis=1)
      else:
          df['SUM'] = df.apply(calculate_custom_sum, axis=1)

    df_ecos['Output'] = df_ecos['SUM'].apply(ecos_rules)
    df_hums['Output'] = df_hums['SUM'].apply(hums_rules)
    df_wealth['Output'] = df_wealth['SUM'].apply(wealth_rules)


    ecos_data = [air_fuzzies, land_fuzzies, water_fuzzies]
    hums_data = [polic_fuzzies, health_fuzzies, know_fuzzies]
    wealth_data = [wealth_pr_fuzzies, wealth_st_fuzzies, wealth_re_fuzzies]

    ecos_ling = pd.DataFrame(ecos_data, columns=VBBAGVG)
    hums_ling = pd.DataFrame(hums_data, columns=VBBAGVG)
    wealth_ling = pd.DataFrame(wealth_data, columns=VBBAGVG)

    ecos_fuzzy_input = dict(ecos_ling)
    hums_fuzzy_input = dict(hums_ling)
    wealth_fuzzy_input = dict(wealth_ling)

    def calculate_primary(fuzzy_input, rule_base):
        terms = max(len(value) for value in fuzzy_input.values())
        for i in range(terms):
            VB_value = fuzzy_input['VB'][i]
            B_value = fuzzy_input['B'][i]
            A_value = fuzzy_input['A'][i]
            G_value = fuzzy_input['G'][i]
            VG_value = fuzzy_input['VG'][i]
            rule_base.iloc[:, i] = rule_base.iloc[:, i].replace(
                {'VB': VB_value, 'B': B_value, 'A': A_value, 'G': G_value, 'VG': VG_value})

        rule_base['Product'] = rule_base.iloc[:, :terms].apply(lambda row: row.product(), axis=1)
        nonzero_products = rule_base[rule_base['Product'] != 0]['Product']
        VB = nonzero_products[rule_base['Output'] == 'VB'].sum()
        B = nonzero_products[rule_base['Output'] == 'B'].sum()
        A = nonzero_products[rule_base['Output'] == 'A'].sum()
        G = nonzero_products[rule_base['Output'] == 'G'].sum()
        VG = nonzero_products[rule_base['Output'] == 'VG'].sum()
        crisp_value = (0 * VB + 0.25 * B + 0.5 * A + 0.75 * G + 1 * VG) / (VB + B + A + G + VG)
        return [crisp_value, [VB, B, A, G, VG]]

    ecos_crisp_value, ecos_fuzzies = calculate_primary(ecos_fuzzy_input, df_ecos)
    hums_crisp_value, hums_fuzzies = calculate_primary(hums_fuzzy_input, df_hums)
    wealth_crisp_value, wealth_fuzzies = calculate_primary(wealth_fuzzy_input, df_wealth)

    for i, df in enumerate(dfs):
        dfs[i] = df[df['Product'] != 0]

    df_osus = generate_combinations(VBBAGVG, 3)

    def osus_rules(row):
        if 0 <= row <= 1:
            return 'EL'
        elif 2 <= row <= 3:
            return 'VL'
        elif row == 4:
            return 'L'
        elif row == 5:
            return 'FL'
        elif row == 6:
            return 'I'
        elif row == 7:
            return 'FH'
        elif row == 8:
            return 'H'
        elif 9 <= row <= 10:
            return 'VH'
        elif 11<= row <= 12:
            return 'EH'

    def calculate_osus_sum(row):
        return sum(row.map({'VB': 0, 'B': 1, 'A': 2, 'G': 3, 'VG': 4}))

    df_osus['SUM'] = df_osus.apply(calculate_osus_sum, axis=1)
    df_osus['Output'] = df_osus['SUM'].apply(osus_rules)

    osus_data = [ecos_fuzzies, hums_fuzzies]
    osus_ling = pd.DataFrame(osus_data, columns=VBBAGVG)
    osus_fuzzy_input = dict(osus_ling)

    def calculate_osus(fuzzy_input, rule_base):
        terms = max(len(value) for value in fuzzy_input.values())
        for i in range(terms):
            VB_value = fuzzy_input['VB'][i]
            B_value = fuzzy_input['B'][i]
            A_value = fuzzy_input['A'][i]
            G_value = fuzzy_input['G'][i]
            VG_value = fuzzy_input['VG'][i]
            rule_base.iloc[:, i] = rule_base.iloc[:, i].replace(
                {'VB': VB_value, 'B': B_value, 'A': A_value, 'G': G_value, 'VG': VG_value})

        rule_base['Product'] = rule_base.iloc[:, :terms].apply(lambda row: row.product(), axis=1)
        nonzero_products = rule_base[rule_base['Product'] != 0]['Product']
        EL = nonzero_products[rule_base['Output'] == 'EL'].sum()
        VL = nonzero_products[rule_base['Output'] == 'VL'].sum()
        L = nonzero_products[rule_base['Output'] == 'L'].sum()
        FL = nonzero_products[rule_base['Output'] == 'FL'].sum()
        I = nonzero_products[rule_base['Output'] == 'I'].sum()
        FH = nonzero_products[rule_base['Output'] == 'FH'].sum()
        H = nonzero_products[rule_base['Output'] == 'H'].sum()
        VH = nonzero_products[rule_base['Output'] == 'VH'].sum()
        EH = nonzero_products[rule_base['Output'] == 'EH'].sum()
        crisp_value = (0 * EL + 0.125 * VL + 0.25 * L + 0.375 * FL + 0.5 * I + 0.625 * FH + 0.75 * H + 0.875 * VH + 1 * EH) / (
                    EL + VL + L + FL + I + FH + H + VH + EH)
        return [crisp_value, [EL, VL, L, FL, I, FH, H, VH, EH]]

    osus_data = [ecos_fuzzies, hums_fuzzies, wealth_fuzzies]
    osus_ling = pd.DataFrame(osus_data, columns=VBBAGVG)
    osus_fuzzy_input = dict(osus_ling)
    osus_crisp_value, osus_fuzzies = calculate_osus(osus_fuzzy_input, df_osus)
    return osus_crisp_value

get_osus(df_xc)

0.45129919287043596

# Sensitivity Analysis

In [ ]:
def sensitivity_analysis(df_xc, delta):
    sensitivity_results = []

    for indicator in range(len(df_xc)):
        # Perturb the specific indicator
        perturbed_df_xc = df_xc.copy()
        xc_plus_delta = min(1.0, df_xc.iloc[indicator] * (1+delta))
        perturbed_df_xc.iloc[indicator] = xc_plus_delta

        # Calculate OSUS for perturbed data
        perturbed_osus = get_osus(perturbed_df_xc)

        # Calculate the gradient using forward difference
        original_osus = get_osus(df_xc)
        delta_c = perturbed_osus - original_osus

        # Calculate Dc value
        Dc = (1 - df_xc.iloc[indicator]) * delta_c
        abs_Dc = abs(Dc)

        # Store the results
        sensitivity_results.append({
            'indicator_index': indicator,
            'xc': df_xc.iloc[indicator],
            'xc+delta': xc_plus_delta,
            'perturbed_osus': perturbed_osus,
            'delta_c': delta_c,
            'Dc': Dc,
            '|Dc|': abs_Dc
        })

    # Create a DataFrame from the results
    sensitivity_df = pd.DataFrame(sensitivity_results)

    # Rank the indicators based on Dc values
    # sensitivity_df = sensitivity_df.sort_values(by='|Dc|', ascending=False).reset_index(drop=True)

    return sensitivity_df

In [ ]:
# +10% perturbation
results_df = sensitivity_analysis(df_xc, delta=0.1)
print(results_df)

    indicator_index        xc  xc+delta  perturbed_osus   delta_c        Dc  \
0                 0  0.945843  1.000000        0.459221  0.007922  0.000429   
1                 1  0.963758  1.000000        0.456955  0.005655  0.000205   
2                 2  0.977564  1.000000        0.454927  0.003628  0.000081   
3                 3  0.968603  1.000000        0.453274  0.001975  0.000062   
4                 4  0.635294  0.698824        0.453004  0.001705  0.000622   
5                 5  0.374674  0.412141        0.452903  0.001604  0.001003   
6                 6  0.798662  0.878528        0.462266  0.010967  0.002208   
7                 7  0.014773  0.016250        0.451386  0.000087  0.000086   
8                 8  0.772414  0.849655        0.462130  0.010831  0.002465   
9                 9  0.319672  0.351639        0.455046  0.003747  0.002549   
10               10  0.965000  1.000000        0.458333  0.007034  0.000246   
11               11  0.586957  0.645652        0.454

In [ ]:
# -10% perturbation
results_df_2 = sensitivity_analysis(df_xc, delta=-0.1)
print(results_df_2)

    indicator_index        xc  xc+delta  perturbed_osus   delta_c        Dc  \
0                 0  0.945843  0.851259        0.437464 -0.013835 -0.000749   
1                 1  0.963758  0.867382        0.436260 -0.015039 -0.000545   
2                 2  0.977564  0.879808        0.435492 -0.015808 -0.000355   
3                 3  0.968603  0.871743        0.445206 -0.006094 -0.000191   
4                 4  0.635294  0.571765        0.449595 -0.001705 -0.000622   
5                 5  0.374674  0.337206        0.449696 -0.001604 -0.001003   
6                 6  0.798662  0.718796        0.440333 -0.010967 -0.002208   
7                 7  0.014773  0.013295        0.451212 -0.000087 -0.000086   
8                 8  0.772414  0.695172        0.440960 -0.010339 -0.002353   
9                 9  0.319672  0.287705        0.447552 -0.003747 -0.002549   
10               10  0.965000  0.868500        0.431906 -0.019393 -0.000679   
11               11  0.586957  0.528261        0.447

In [ ]:
# Export the dfs to the google sheet
sheet2 = wb.worksheet("SA")
sheet2.update([results_df.columns.values.tolist()] + results_df.values.tolist())
sheet2.update('I1', [results_df_2.columns.values.tolist()] + results_df_2.values.tolist())

{'spreadsheetId': '169n2zgG-2fvAhgrbeLBlaQ8vZXMPlQAik5P7yxjxvGA',
 'updatedRange': 'SA!I1:O23',
 'updatedRows': 23,
 'updatedColumns': 7,
 'updatedCells': 161}